# Using the XApRES package
A guide to using the package XApRES to load and package together timeseries of ApRES datasets. 

## Load the package and set up workspace

In [1]:
import sys
import os
from pathlib import Path
path = Path(sys.path[0])
sys.path.append(os.path.join(path.parent.parent.absolute(),'src','XApRES'))
import ApRESDefs
import numpy as np
import matplotlib.pyplot as plt


In [2]:
xa = ApRESDefs.xapres(loglevel='debug')

DEBUG    18:29:07 	 ApRESDefs.py @function _setup_logging line 494 - Stream logging level set to DEBUG
DEBUG    18:29:07 	 ApRESDefs.py @function _setup_logging line 495 - Add console handler to logger
DEBUG    18:29:07 	 ApRESDefs.py @function _setup_logging line 508 - File logging level set to DEBUG


In [ ]:
filepaths = xa.list_files(directory='gs://ldeo-glaciology/GL_apres_2022', remote_load = True)
filepaths[0:5]

DEBUG    18:29:07 	 ApRESDefs.py @function list_files line 165 - Find all the dat files in the directory gs://ldeo-glaciology/GL_apres_2022 with remote_load = True


_request out of retries on exception: Cannot connect to host storage.googleapis.com:443 ssl:default [nodename nor servname provided, or not known]
Traceback (most recent call last):
  File "/Users/georgelu/opt/anaconda3/envs/keras/lib/python3.10/site-packages/aiohttp/connector.py", line 1154, in _create_direct_connection
    hosts = await asyncio.shield(host_resolved)
  File "/Users/georgelu/opt/anaconda3/envs/keras/lib/python3.10/site-packages/aiohttp/connector.py", line 880, in _resolve_host
    addrs = await self._resolver.resolve(host, port, family=self._family)
  File "/Users/georgelu/opt/anaconda3/envs/keras/lib/python3.10/site-packages/aiohttp/resolver.py", line 33, in resolve
    infos = await self._loop.getaddrinfo(
  File "/Users/georgelu/opt/anaconda3/envs/keras/lib/python3.10/asyncio/base_events.py", line 860, in getaddrinfo
    return await self.run_in_executor(
  File "/Users/georgelu/opt/anaconda3/envs/keras/lib/python3.10/concurrent/futures/thread.py", line 58, in run
 

## Load 3 Bursts

In [ ]:
%%time 
import importlib
importlib.reload(ApRESDefs)  
xa = ApRESDefs.xapres(loglevel='debug', max_range=1400)
xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022', 
            remote_load = True,
            file_numbers_to_process = [1,2,3,200], 
            bursts_to_process=[0,1]
           )
xa.data

## Stack bursts

In [ ]:
# stack the data (this line stacks both the chirps and the profiles, but you could just do the profiles)
stacked1 = xa.data.isel(time=0, attenuator_setting_pair=0).mean(dim='chirp_num')
stacked2 = xa.data.isel(time=1, attenuator_setting_pair=0).mean(dim='chirp_num')
stacked3 = xa.data.isel(time=6, attenuator_setting_pair=0).mean(dim='chirp_num')


xa.dB(stacked1.profile).plot(label=stacked1.profile.time.data)
xa.dB(stacked2.profile).plot(label=stacked2.profile.time.data)
xa.dB(stacked3.profile).plot(label=stacked3.profile.time.data)
plt.legend()
plt.title('Range Profiles')


## Calculate vertical velocities for a pair of stacked bursts
The function looks like `generate_range_diff(self, data1, data2, win_cor, step, range_ext=None, win_wrap=10, thresh=0.9, uncertainty='noise_phasor')`. Changing the window for the coherence and the step creates pretty different results.

In [ ]:
for step in [10,20]:
    for win in [10,20,30]:
        vels, ds = xa.generate_range_diff(stacked1.profile,stacked2.profile,win,step)
        plt.plot(ds,vels,label=f'Coherence window: {win}, Coherence step: {step}')
plt.xlabel('depth [m]')
plt.ylabel('vertical velocity [m/s]')
plt.legend()

With times split further apart, this windowing and step size has even more impact. 

In [ ]:
for step in [10,20]:
    for win in [10,20,30]:
        vels, ds = xa.generate_range_diff(stacked1.profile,stacked3.profile,win,step)
        plt.plot(ds,vels,label=f'Coherence window: {win}, Coherence step: {step}')
plt.xlabel('depth [m]')
plt.ylabel('vertical velocity [m/s]')
plt.legend()

## Testing with multiple bursts in data

Given the example below, clearly at the moment something causes the range_diff calculation to collapse, perhaps it doesn't like working with dimensions. A possible solution is to have a for loop iterating through each pair to compare. This is simple to implement, but could be annoying with big data (though I see that in generating the xarrays, there's for loops anyways). Another solution that I need to sleep on is to modify the calculations form ImpDAR to accomodate the xarray structure (since currently it is only operating with numpy arrays). This may be a bit trickier as in I can't think of the immediate implementation. 

In [ ]:
stacked_arr1 = xa.data.isel(time=[0,1,2,3], attenuator_setting_pair=0).mean(dim='chirp_num')
stacked_arr2 = xa.data.isel(time=[1,2,3,4], attenuator_setting_pair=0).mean(dim='chirp_num')

In [ ]:
vels, ds = xa.generate_range_diff(stacked_arr1.profile,stacked_arr2.profile,win,step)

In [ ]:
vels

In [ ]:
ds